# 🤖 PAROL6夹爪测试 - Gripper Test
# 
# 本笔记本用于测试PAROL6机械臂的真实夹爪模型
# This notebook is for testing the PAROL6 robot arm's real gripper model
#
# 测试内容 Test Contents:
# 1. 加载带有真实夹爪的XML模型
#    Load XML model with real gripper
# 2. 可视化机械臂和夹爪
#    Visualize robot arm and gripper
# 3. 测试夹爪开合控制
#    Test gripper open/close control
# 4. 测试机械臂6个关节控制
#    Test 6-DOF arm control

In [ ]:
# ========================================
# Cell 1 - 环境变量设置和工作目录
# ========================================

import os

# 设置工作目录以确保相对路径正确
os.chdir("/home/wzy/lerobot-mujoco")
print(f"✅ 工作目录设置为: {os.getcwd()}")

# 设置X11显示服务器
os.environ['DISPLAY'] = ':0'
os.environ['XAUTHORITY'] = os.path.expanduser('~/.Xauthority')
print(f"✓ DISPLAY设置为: {os.environ['DISPLAY']}")

# 强制使用GPU渲染
os.environ['MUJOCO_GL'] = 'egl'
print(f"✓ MUJOCO_GL: egl (GPU硬件加速)")

# NVIDIA GPU渲染优化
os.environ['__GL_SYNC_TO_VBLANK'] = '0'
os.environ['__GL_YIELD'] = 'NOTHING'
print("✓ NVIDIA GPU优化已启用")

# OpenGL性能优化
os.environ['__GL_FSAA_MODE'] = '0'
os.environ['__GL_LOG_MAX_ANISO'] = '0'
print("✓ OpenGL性能优化已启用")

print("\n🎉 所有环境变量设置完成！")

In [ ]:
# ========================================
# Cell 2 - 导入必要的库
# ========================================

import mujoco
import numpy as np
import time
from PIL import Image
import matplotlib.pyplot as plt

print("✅ 所有库导入成功！")
print(f"   MuJoCo版本: {mujoco.__version__}")

In [ ]:
# ========================================
# Cell 3 - 加载PAROL6模型（带真实夹爪）
# ========================================

# XML文件路径 - 使用绝对路径避免相对路径解析问题
xml_path = '/home/wzy/lerobot-mujoco/01-Parol6/parol6_full.xml'

print("📖 加载MuJoCo模型...")
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)

print("✅ 模型加载成功！")
print(f"   总关节数: {model.njnt}")
print(f"   总执行器: {model.nu}")
print(f"   总body数: {model.nbody}")
print(f"   摄像头数: {model.ncam}")

# 打印所有关节名称
print("\n🔧 关节列表:")
for i in range(model.njnt):
    joint_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)
    if joint_name:
        joint_type = 'revolute' if model.jnt_type[i] == 0 else 'slide'
        print(f"   {i}: {joint_name} ({joint_type})")

# 打印所有摄像头
print("\n📷 摄像头列表:")
for i in range(model.ncam):
    cam_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_CAMERA, i)
    print(f"   {i}: {cam_name}")

In [ ]:
# ========================================
# Cell 4 - 渲染和可视化函数
# ========================================

def render_camera(model, data, camera_name="agentview", width=640, height=480):
    """
    从指定摄像头渲染图像
    
    Args:
        model: MuJoCo模型
        data: MuJoCo数据
        camera_name: 摄像头名称
        width: 图像宽度
        height: 图像高度
    
    Returns:
        numpy array: RGB图像 (H, W, 3)
    """
    # 创建渲染器
    renderer = mujoco.Renderer(model, height=height, width=width)
    
    # 更新场景
    renderer.update_scene(data, camera=camera_name)
    
    # 渲染图像
    image = renderer.render()
    
    return image

def show_multi_view(model, data):
    """
    显示多视角图像
    """
    # 获取所有摄像头的图像
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    camera_names = ['agentview', 'gripper_cam', 'topview', 'sideview']
    
    for idx, cam_name in enumerate(camera_names):
        try:
            image = render_camera(model, data, camera_name=cam_name)
            ax = axes[idx // 2, idx % 2]
            ax.imshow(image)
            ax.set_title(f'{cam_name}', fontsize=14, fontweight='bold')
            ax.axis('off')
        except Exception as e:
            print(f"⚠️ 无法渲染摄像头 {cam_name}: {e}")
    
    plt.tight_layout()
    plt.show()

print("✅ 渲染函数定义完成！")

In [ ]:
# ========================================
# Cell 5 - 初始状态可视化
# ========================================

# 重置到初始状态
mujoco.mj_resetData(model, data)

# 前向运动学
mujoco.mj_forward(model, data)

print("📸 初始状态多视角图像:")
show_multi_view(model, data)

# 打印初始关节位置
print("\n🔧 初始关节位置:")
for i in range(model.nq):
    print(f"   qpos[{i}] = {data.qpos[i]:.4f}")

In [ ]:
# ========================================
# Cell 6 - 测试夹爪开合（闭合）
# ========================================

print("🤏 测试夹爪闭合...")

# 重置模型
mujoco.mj_resetData(model, data)

# 设置夹爪控制量
# data.ctrl[6] = 左手指 (rh_l1)
# data.ctrl[7] = 右手指 (rh_r1)
# 值范围：[0, 0.03]，0表示闭合，0.03表示完全打开

# 闭合夹爪（设置为0）
data.ctrl[6] = 0.0  # 左手指闭合
data.ctrl[7] = 0.0  # 右手指闭合

# 运行仿真让夹爪闭合
for _ in range(200):
    mujoco.mj_step(model, data)

print("✅ 夹爪已闭合")
print(f"   左手指位置: {data.qpos[6]:.4f}")
print(f"   右手指位置: {data.qpos[7]:.4f}")

# 显示闭合状态
print("\n📸 夹爪闭合状态:")
show_multi_view(model, data)

In [ ]:
# ========================================
# Cell 7 - 测试夹爪开合（打开）
# ========================================

print("✋ 测试夹爪打开...")

# 重置模型
mujoco.mj_resetData(model, data)

# 打开夹爪（设置为最大值）
data.ctrl[6] = 0.03  # 左手指打开
data.ctrl[7] = 0.03  # 右手指打开

# 运行仿真让夹爪打开
for _ in range(200):
    mujoco.mj_step(model, data)

print("✅ 夹爪已打开")
print(f"   左手指位置: {data.qpos[6]:.4f}")
print(f"   右手指位置: {data.qpos[7]:.4f}")

# 显示打开状态
print("\n📸 夹爪打开状态:")
show_multi_view(model, data)

In [ ]:
# ========================================
# Cell 8 - 测试机械臂关节控制
# ========================================

print("🦾 测试机械臂6个关节控制...")

# 重置模型
mujoco.mj_resetData(model, data)

# 设置目标关节位置（弧度）
target_positions = [0.5, -0.3, 0.8, -1.2, 0.6, 0.4]  # 6个关节的目标位置

# 应用控制信号
for i in range(6):
    data.ctrl[i] = target_positions[i]

# 运行仿真让机械臂运动到目标位置
for _ in range(500):
    mujoco.mj_step(model, data)

print("✅ 机械臂已运动到目标位置")
print("\n🔧 最终关节位置:")
for i in range(6):
    joint_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)
    print(f"   {joint_name}: {data.qpos[i]:.4f} rad")

# 显示最终状态
print("\n📸 机械臂目标位置状态:")
show_multi_view(model, data)

In [ ]:
# ========================================
# Cell 9 - 测试组合动作（机械臂+夹爪）
# ========================================

print("🎯 测试组合动作：机械臂运动 + 夹爪开合...")

# 重置模型
mujoco.mj_resetData(model, data)

# 设置机械臂目标位置
target_positions = [0.3, -0.5, 0.6, -0.8, 0.4, 0.2]
for i in range(6):
    data.ctrl[i] = target_positions[i]

# 设置夹爪半开状态
data.ctrl[6] = 0.015  # 左手指半开
data.ctrl[7] = 0.015  # 右手指半开

# 运行仿真
for _ in range(500):
    mujoco.mj_step(model, data)

print("✅ 组合动作完成！")
print(f"   夹爪位置: 左手指={data.qpos[6]:.4f}, 右手指={data.qpos[7]:.4f}")

# 显示最终状态
print("\n📸 组合动作最终状态:")
show_multi_view(model, data)

In [ ]:
# ========================================
# Cell 10 - 总结
# ========================================

print("🎉 PAROL6机械臂夹爪测试完成！")
print("=====================================")
print("✅ 成功加载了带有真实夹爪的PAROL6模型")
print("✅ 夹爪开合控制正常工作")
print("✅ 机械臂6个关节控制正常")
print("✅ 多视角可视化功能正常")
print("✅ 组合动作（机械臂+夹爪）协调工作")
print("=====================================")
print("\n📊 模型统计信息:")
print(f"   总关节数: {model.njnt}")
print(f"   总执行器: {model.nu}")
print(f"   总body数: {model.nbody}")
print(f"   摄像头数: {model.ncam}")
print(f"   夹爪关节: 2个（左右手指）")
print(f"   机械臂关节: 6个（L1-L6）")
print("\n🔧 关节详细信息:")
for i in range(model.njnt):
    joint_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)
    if joint_name:
        joint_type = 'revolute' if model.jnt_type[i] == 0 else 'slide'
        print(f"   {joint_name}: {joint_type}")